# Opioid Prescriptions

In [1]:
# Depedencies
import pandas as pd

In [283]:
# Combine raw prescription data 2013-2017 into one pandas dataframe

# Columns pulled from raw data:
# nppes_provider_city: City of the Provider
# nppes_provider_state: State Code of the Provider
# opioid_claim_count: Total Claims of Opioid Drugs, Including Refills
# opioid_day_supple: Number of Day's Supply of All Opioid Drugs
# opioid_prescriber_rate: Percent of Total_Claim_Count Represented by Opioid_Claim_Count
# la_opioid tags follow the same as above

# Loop through each raw data file and append to dataframe
prescriptions_df = pd.DataFrame()
x = 13
year = 2013

while x <= 17:
    
    # Change file path for years 13-17 and read into pandas
    file_path = f'prescriptions/raw_data/PartD_Prescriber_PUF_NPI_{x}.txt'
    df = pd.read_table(file_path)
    
    # Select only the columns of interest (see above)
    df = df[[
            'nppes_provider_city',
            'nppes_provider_state',
            'opioid_claim_count', 
            'opioid_day_supply', 
            'opioid_prescriber_rate', 
            'la_opioid_claim_count', 
            'la_opioid_day_supply',
            'la_opioid_prescriber_rate',
            ]]
    
    # Rename columns
    df.rename(columns={'nppes_provider_city': 'city', 'nppes_provider_state': 'state'}, inplace=True)
    
    # Fill in 0 for NA values
    df.fillna(0, inplace=True)
    
    # Add year column for current file year
    df['year'] = year
    
    # Append to main dataframe
    prescriptions_df = prescriptions_df.append(df)
    
    # Increment file year name and column year value
    x += 1
    year += 1
    
    continue

In [284]:
# Sum the claim counts and day supplies, average the prescriber rates
prescriptions_df = prescriptions_df.groupby(['state','city', 'year']).agg({
        'opioid_claim_count':'sum',
        'opioid_day_supply':'sum',
        'opioid_prescriber_rate':'mean',
        'la_opioid_claim_count':'sum',
        'la_opioid_day_supply':'sum',
        'la_opioid_prescriber_rate':'mean'
        }).reset_index()

In [285]:
# Import county info file for linking city+state to county
city_county_df = pd.read_csv('prescriptions/ZIP-COUNTY-FIPS_2018-03.csv')

# Pull columns of interest
city_county_df = city_county_df[['STCOUNTYFP', 'COUNTYNAME', 'CITY', 'STATE']]

# Rename columns
city_county_df.rename(columns={
        'STCOUNTYFP':'county_FIPS', 
        'COUNTYNAME':'county', 
        "CITY":'city', 
        'STATE':'state'
        }, inplace=True)

# Match city data type and caps for both datasets to prepare for join
city_county_df['city'] = city_county_df['city'].str.upper()
prescriptions_df['city'] = prescriptions_df['city'].str.upper()

# Merge the data to add county name and FIPS code to prescription dataframe
prescriptions_df = pd.merge(prescriptions_df, city_county_df, on=['city', 'state'], how='inner')

In [286]:
# Sum the claim counts and day supplies, average the prescriber rates
prescriptions_df = prescriptions_df.groupby(['state', 'county', 'county_FIPS', 'year']).agg({
        'opioid_claim_count':'sum',
        'opioid_day_supply':'sum',
        'opioid_prescriber_rate':'mean',
        'la_opioid_claim_count':'sum',
        'la_opioid_day_supply':'sum',
        'la_opioid_prescriber_rate':'mean'
        }).reset_index()

In [290]:
prescriptions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15975 entries, 0 to 15974
Data columns (total 10 columns):
state                        15975 non-null object
county                       15975 non-null object
county_FIPS                  15975 non-null int64
year                         15975 non-null int64
opioid_claim_count           15975 non-null float64
opioid_day_supply            15975 non-null float64
opioid_prescriber_rate       15975 non-null float64
la_opioid_claim_count        15975 non-null float64
la_opioid_day_supply         15975 non-null float64
la_opioid_prescriber_rate    15975 non-null float64
dtypes: float64(6), int64(2), object(2)
memory usage: 1.2+ MB


In [291]:
prescriptions_df.head()

,state,county,county_FIPS,year,opioid_claim_count,opioid_day_supply,opioid_prescriber_rate,la_opioid_claim_count,la_opioid_day_supply,la_opioid_prescriber_rate
0,AK,Aleutians East Borough,2013,2013,42.0,787.0,5.960000,0.0,0.0,0.0
1,AK,Aleutians East Borough,2013,2014,83.0,1947.0,5.420000,0.0,0.0,0.0
2,AK,Aleutians East Borough,2013,2015,29.0,624.0,0.586667,0.0,0.0,0.0
3,AK,Aleutians East Borough,2013,2016,66.0,1801.0,1.965000,0.0,0.0,0.0
4,AK,Aleutians East Borough,2013,2017,54.0,1186.0,3.060000,0.0,0.0,0.0


In [293]:
# Save the dataframe as CSV
prescriptions_df.to_csv('prescriptions/prescriptions.csv', index=False)